In [1]:
from datasets import load_dataset

/home/vojtasek/equi-act/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("dpdl-benchmark/colorectal_histology")


In [4]:
seen_labels = []
for element in ds["train"]:
    if element["label"] not in seen_labels:
        print(element["label"])
        seen_labels.append(element["label"])

4
5
0
6
1
3
7
2


In [18]:
from escnn import nn, gspaces

o2 = gspaces.flipRot2dOnR2(maximum_frequency=3)
o2.irreps
so2 = gspaces.rot2dOnR2(maximum_frequency=3)
so2.irreps

[SO(2)|[irrep_0]:1, SO(2)|[irrep_1]:2, SO(2)|[irrep_2]:2, SO(2)|[irrep_3]:2]

In [29]:
import numpy as np
thetas = np.linspace(0.0, 2*np.pi, 8, endpoint=False)
print(thetas)
o2.fibergroup.element((0,thetas[1]))

[0.         0.78539816 1.57079633 2.35619449 3.14159265 3.92699082
 4.71238898 5.49778714]


(+, 0.7853981633974483)

In [2]:
from datasets import load_dataset

ds = load_dataset("blanchon/EuroSAT_RGB")



Generating validation split: 100%|██████████| 5400/5400 [00:00<00:00, 49654.25 examples/s]


In [ ]:
train_split = ds["train"]
val_split = ds["validation"]
test_split = ds["test"]

In [ ]:
len(train_split), len(val_split), len(test_split)

(16200, 5400, 5400)

In [10]:
import os
from typing import Tuple
from pathlib import Path
import lightning as L
import numpy as np
import torch
from PIL import Image
from datasets import load_dataset, ClassLabel
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, Pad, Resize, RandomRotation, InterpolationMode, ToTensor, RandomVerticalFlip, RandomHorizontalFlip
from torch_geometric.transforms import NormalizeScale, SamplePoints
from torch_geometric.datasets import ModelNet
from torch_geometric.loader import DataLoader

class ModelNet10PointClouds(L.LightningDataModule):
    def __init__(
        self,
        root: str = "./data/modelnet10",
        num_points: int = 2048,
        batch_size: int = 32,
        num_workers: int = 4,
        val_split: float = 0.1,
        shuffle_points: bool = True,
    ):
        super().__init__()
        self.root = root
        self.num_points = num_points
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_split = val_split
        self.shuffle_points = shuffle_points

        # Point-cloud transforms: normalize to unit sphere and sample N points per mesh
        self.transform = Compose([
            NormalizeScale(),
            SamplePoints(self.num_points, remove_faces=True),
        ])

    def prepare_data(self):
        # This triggers the download once; PyG handles the URL/zip/unpack for ModelNet10.
        ModelNet(self.root, name="10", train=True)
        ModelNet(self.root, name="10", train=False)

    def setup(self, stage=None):
        train_full = ModelNet(self.root, name="10", train=True, transform=self.transform)
        test = ModelNet(self.root, name="10", train=False, transform=self.transform)

        # Split train into train/val
        n_total = len(train_full)
        n_val = int(self.val_split * n_total)
        n_train = n_total - n_val
        self.train_set, self.val_set = random_split(train_full, [n_train, n_val],
                                                    generator=torch.Generator().manual_seed(42))
        self.test_set = test

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size,
                          shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size,
                          shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size,
                          shuffle=False, num_workers=self.num_workers)
    

dm = ModelNet10PointClouds(num_points=2048, batch_size=16)
dm.prepare_data()
dm.setup()

# Peek at one batch
batch = next(iter(dm.train_dataloader()))
# batch is a PyG Batch with fields like .pos (B*N x 3), .y (labels), and .batch (graph ids)
print("pos shape:", batch.pos.shape)     # (total_points_in_batch, 3)
print("labels shape:", batch.y.shape)    # (batch_size,)
print("num graphs:", batch.num_graphs)


Extracting data/modelnet10/ModelNet10.zip
Processing...
Done!


pos shape: torch.Size([32768, 3])
labels shape: torch.Size([16])
num graphs: 16
